In [1]:
from huggingface_hub import login
from datasets import load_dataset
import os

login(os.getenv("hf_token"))

train = load_dataset("dragonslayer631/ci2_allsides", split="train")
test = load_dataset("dragonslayer631/ci2_allsides", split="test")

train = train.to_pandas()
test = test.to_pandas()

/Users/upasana/code/playground/ci2_proj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
import pandas as pd
from sklearn.metrics import classification_report

In [52]:
joined = pd.concat([train, test], axis=0)

joined.shape

(45089, 47)

In [53]:
joined.columns

Index(['topic', 'tags', 'text', 'int_bias', 'summary_5', 'summary_50',
       'summary_100', 'text_entity_sentiments', 'text_topic_to_sentiment',
       'summary_100_entity_sentiments', 'summary_100_topic_to_sentiment',
       'summary_50_entity_sentiments', 'summary_50_topic_to_sentiment', 'id',
       'text_encoded', 'summary_100_encoded', 'summary_50_encoded',
       'text_topic_0', 'text_sentiment_0', 'text_topic_1', 'text_sentiment_1',
       'text_topic_2', 'text_sentiment_2', 'text_topic_3', 'text_sentiment_3',
       'text_topic_4', 'text_sentiment_4', 'summary_100_topic_0',
       'summary_100_sentiment_0', 'summary_100_topic_1',
       'summary_100_sentiment_1', 'summary_100_topic_2',
       'summary_100_sentiment_2', 'summary_100_topic_3',
       'summary_100_sentiment_3', 'summary_100_topic_4',
       'summary_100_sentiment_4', 'summary_50_topic_0',
       'summary_50_sentiment_0', 'summary_50_topic_1',
       'summary_50_sentiment_1', 'summary_50_topic_2',
       'summary_

In [ ]:
from utils.df_utils import expand_columns, keep_columns_related_to

In [ ]:
joined = keep_columns_related_to(joined, "text")

In [ ]:
joined = expand_columns(
    joined, 
    topic_columns=['text_topic_0', 'text_topic_1', 'text_topic_2', 'text_topic_3', 'text_topic_4'], 
    sentiment_columns=['text_sentiment_0', 'text_sentiment_1', 'text_sentiment_2', 'text_sentiment_3', 'text_sentiment_4'], 
    encoded_text_column='text_encoded'
)

In [60]:
# split back into test and train

train_idx = train.index
test_idx = test.index

X_train = joined.loc[train_idx]
Y_train = X_train[['int_bias']]
X_train.drop(["int_bias"], axis=1, inplace=True)

X_test = joined.loc[test_idx]
Y_test = X_test[['int_bias']]
X_test.drop(["int_bias"], axis=1, inplace=True)

X_train = X_train.convert_dtypes()
X_test = X_test.convert_dtypes()
Y_test = Y_test.convert_dtypes()
Y_train = Y_train.convert_dtypes()



print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(36071, 5658) (36071, 1) (9018, 5658) (9018, 1)


In [61]:
from xgboost import XGBClassifier

model = XGBClassifier(
    objective='multi:softprob',  # outputs class probabilities
    num_class=3,
    max_depth=8,                # Avoid extreme depth like 128
    tree_method='hist',     # Use GPU if available
    predictor='gpu_predictor',  # Use GPU for inference
    use_label_encoder=False,    # suppress warning
    eval_metric='mlogloss',
    device=device
)

model.fit(X_train, Y_train)

# # Predict probabilities for each class
# probs = model.predict_proba(X_test_updated)

# Predict class labels
preds = model.predict(X_test)

print(classification_report(Y_test, preds, digits=4))

/Users/upasana/code/playground/ci2_proj/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:28:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

         0.0     1.0000    0.9997    0.9999      3677
         1.0     0.9995    1.0000    0.9998      2208
         2.0     1.0000    1.0000    1.0000      3133

    accuracy                         0.9999      9018
   macro avg     0.9998    0.9999    0.9999      9018
weighted avg     0.9999    0.9999    0.9999      9018



In [62]:
model.save_model("models/xgb_text_softprob.json")  # or .bin